In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
import bw2analyzer as ba
import numpy as np

In [2]:
bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")

Restoring project backup archive - this could take a few minutes...


'ei38-teaching-25'

In [3]:
bd.projects.set_current('ei38-teaching-25')

In [4]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei 3.8 cutoff

In [7]:
wood_pellets_heat = bd.get_node(database="ei 3.8 cutoff", name='wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future', unit='megajoule')

In [8]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [9]:
wood_pellets_market = bd.get_node(database="ei 3.8 cutoff", name = "market for wood pellet, measured as dry mass", location = "RER")

In [10]:
wood_pellets_prod_RER = bd.get_node(database="ei 3.8 cutoff", name = "wood pellet production", location = "RER")

In [11]:
if "Regionalized pellets" in bd.databases:
    del bd.databases["Regionalized pellets"]

In [12]:
DB_pellets = bd.Database("Regionalized pellets", backend = "iotable")

In [13]:
DB_pellets.write({})

In [14]:
bd.databases

Databases dictionary with 3 object(s):
	Regionalized pellets
	biosphere3
	ei 3.8 cutoff

In [15]:
new_wood_pellets_heat = DB_pellets.new_node(code="pellets-heat", name="wood pellets, burned in stirling heat and power co-generation unit, 3kW electrical, future - regionalized", location="CH", unit="megajoule")
new_wood_pellets_heat.save()

In [16]:
new_wood_pellets_market = DB_pellets.new_node(code="pellets-market", name="market for wood pellet, measured as dry mass", location="CH", unit="kilogram")
new_wood_pellets_market.save()

In [17]:
new_pellets_prod = DB_pellets.new_node(code="pellets-prod", name="wood pellet production", location="CH", unit="kilogram")
new_pellets_prod.save()

In [19]:
from collections import defaultdict

new_pellets_tech = defaultdict(float)

for exc in wood_pellets_prod_RER.technosphere():
    new_pellets_tech[bd.get_node(
        database = "ei 3.8 cutoff",
        name = exc.input['name'],
        location = exc.input['location']
    )] += exc['amount']

In [20]:
tech_edges = [
    {"row": new_wood_pellets_heat.id, "col": new_wood_pellets_heat.id, "amount":1},
    {"row": wood_pellets_heat.id, "col": new_wood_pellets_heat.id, "amount":1, "flip":True},
    {"row": new_wood_pellets_market.id, "col": new_wood_pellets_heat.id, "amount":0.0229911254795743, "flip":True},
    {"row": wood_pellets_market.id, "col": new_wood_pellets_heat.id, "amount": 0.0229911254795743},
] + [
    {"row": new_wood_pellets_market.id, "col": new_wood_pellets_market.id, "amount":1},
    {"row": wood_pellets_market.id, "col": new_wood_pellets_market.id, "amount":1, "flip":True},
    {"row": new_pellets_prod.id, "col": new_wood_pellets_market.id, "amount": 1, "flip":True},
    {"row": wood_pellets_prod_RER.id, "col": new_wood_pellets_market.id, "amount": 1},
] + [
    {"row": new_pellets_prod.id, "col": new_pellets_prod.id, "amount": 1},
    {"row": wood_pellets_prod_RER.id, "col": new_pellets_prod.id, "amount": 1, "flip":True},
    {
        "row": bd.get_activity(
            database="ei 3.8 cutoff", 
            name='market group for electricity, medium voltage',
            location='RER'
        ).id, 
        "col": new_pellets_prod.id, 
        "amount": 0.096
    },
] + [
    {"row": key.id, "col": new_pellets_prod.id, "amount": value}
    for key, value in new_pellets_tech.items() if key['location']=="Europe without Switzerland"
] + [
    {
        "row": bd.get_activity(
            database="ei 3.8 cutoff", 
            name='market for electricity, medium voltage',
            location='CH'
        ).id, 
        "col": new_pellets_prod.id, 
        "amount": 0.096,
        "flip":True
    },
] + [
    {"row": bd.get_activity(database="ei 3.8 cutoff", name=key['name'], location="CH").id, 
      "col": new_pellets_prod.id, "amount": value, "flip":True}
    for key, value in new_pellets_tech.items() if key['location']=="Europe without Switzerland"

]

In [21]:
DB_pellets.write_exchanges(tech_edges, [], ["ei 3.8 cutoff"])

Starting IO table write
Adding technosphere matrix
Adding biosphere matrix
Finalizing serialization


In [22]:
wood_pellets_heat.new_edge(input=new_wood_pellets_heat, amount=0, type="technosphere").save()

In [23]:
_, data_objs, _ = bd.prepare_lca_inputs({wood_pellets_heat: 1}, ipcc)

In [24]:
lca = bc.LCA({wood_pellets_heat.id: 1}, data_objs=data_objs)
lca.lci()
lca.lcia()
lca.score

0.004655632602942414

In [25]:
lca.lcia({new_wood_pellets_heat.id: 1})
lca.score

0.003645551616133234